# Tutorial: Query Classification with TransformersTextRouter and TransformersZeroShotTextRouter

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Components Used**: [`TransformersZeroShotTextRouter`](https://docs.haystack.deepset.ai/docs/transformerszeroshottextrouter), [`TransformersTextRouter`](https://docs.haystack.deepset.ai/docs/transformerstextrouter), [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [`SentenceTransformersTextEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever)
- **Goal**: After completing this tutorial, you will have learned about TransformersZeroShotTextRouter and TransformersTextRouter and how to use them in a pipeline.

## Overview

One of the great benefits of using state-of-the-art NLP models like those available in Haystack is that it allows users to state their queries as *plain natural language questions*: rather than trying to come up with just the right set of keywords to find the answer to their question, users can simply ask their question in much the same way that they would ask it of a (very knowledgeable!) person.

But just because users *can* ask their questions in "plain English" (or "plain German", etc.), that doesn't mean they always *will*. For instance, users might input a few keywords rather than a complete question because they don't understand the pipeline's full capabilities or are so accustomed to keyword search. While a standard Haystack pipeline might handle such queries with reasonable accuracy, for a variety of reasons we still might prefer that our pipeline is sensitive to the type of query it is receiving, so that it behaves differently when a user inputs, say, a collection of keywords instead of a question. For this reason, Haystack comes with built-in capabilities to distinguish between types of text inputs, such as **keyword queries**, **interrogative queries(questions)**, and **statement queries**.

Given a text input, classification models ouput a label, which can be LABEL_0,
LABEL_1, ... depending on how the model was trained. Haystack's TransformersTextRouter component uses classification models and then routes the text to an output branch that is named after the label.

In this tutorial you will learn how to use TransformersTextRouter and TransformersZeroShotTextRouter to branch your Haystack pipeline based on the type of query it receives:

1. **Keyword vs. Question/Statement** &mdash; routes a query into one of two branches depending on whether it is a full question/statement or a collection of keywords.

2. **Question vs. Statement** &mdash; routes a natural language query into one of two branches depending on whether it is a question or a statement.

With TransformersTextRouter, it's also possible to route queries based on your custom classification models. With TransformersZeroShotTextRouter you can even do zero-shot classification, which means you can specify custom classes but don't need any custom model.

With all of that explanation out of the way, let's dive in!


## Preparing the Colab Environment

- [Enable GPU Runtime](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)


## Installing Haystack

To start, install the latest release of Haystack with `pip`:

In [ ]:
%%bash

pip install --upgrade pip
pip install haystack-ai torch sentencepiece datasets sentence-transformers

### Enabling Telemetry
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(41)

## Trying out TransformersTextRouter

Before integrating a TransformersTextRouter into the pipeline, test it out on its own and see what it actually does. First, initiate a simple, out-of-the-box **keyword vs. question/statement** TransformersTextRouter, which uses the [shahrukhx01/bert-mini-finetune-question-detection](https://huggingface.co/shahrukhx01/bert-mini-finetune-question-detection) model.
For this model, LABEL_0 corresponds to keyword queries and LABEL_1 to non-keyword queries.

In [ ]:
from haystack.components.routers import TransformersTextRouter

text_router = TransformersTextRouter(model="shahrukhx01/bert-mini-finetune-question-detection")
text_router.warm_up()

Now feed some queries into this text router. Test with one keyword query, one interrogative query, and one statement query. Note that you don't need to use any punctuation, such as question marks, for the text router to make the right decision.

In [4]:
queries = [
    "Arya Stark father",  # Keyword Query
    "Who was the father of Arya Stark",  # Interrogative Query
    "Lord Eddard was the father of Arya Stark",  # Statement Query
]

Below, you can see what the text router does with these queries: it correctly determines that  "Arya Stark father" is a keyword query and sends it to the branch LABEL_0. It also correctly classifies both the interrogative query "Who was the father of Arya Stark" and the statement query "Lord Eddard was the father of Arya Stark" as non-keyword queries, and sends them to branch 1.

In [5]:
result = text_router.run(text=queries[0])
next(iter(result))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


'LABEL_0'

In [6]:
import pandas as pd

results = {"Query": [], "Output Branch": [], "Class": []}

for query in queries:
    result = text_router.run(text=query)
    results["Query"].append(query)
    results["Output Branch"].append(next(iter(result)))
    results["Class"].append("Keyword Query" if next(iter(result)) == "LABEL_0" else "Question/Statement")

pd.DataFrame.from_dict(results)

,Query,Output Branch,Class
0,Arya Stark father,LABEL_0,Keyword Query
1,Who was the father of Arya Stark,LABEL_1,Question/Statement
2,Lord Eddard was the father of Arya Stark,LABEL_1,Question/Statement


Next, you will illustrate a **question vs. statement** TransformersTextRouter using [shahrukhx01/question-vs-statement-classifier](https://huggingface.co/shahrukhx01/question-vs-statement-classifier). For this task, you need to initialize a new text router with this classification model.

In [7]:
text_router = TransformersTextRouter(model="shahrukhx01/question-vs-statement-classifier")
text_router.warm_up()

queries = [
    "Who was the father of Arya Stark",  # Interrogative Query
    "Lord Eddard was the father of Arya Stark",  # Statement Query
]

results = {"Query": [], "Output Branch": [], "Class": []}

for query in queries:
    result = text_router.run(text=query)
    results["Query"].append(query)
    results["Output Branch"].append(next(iter(result)))
    results["Class"].append("Question" if next(iter(result)) == "LABEL_1" else "Statement")

pd.DataFrame.from_dict(results)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,Query,Output Branch,Class
0,Who was the father of Arya Stark,LABEL_1,Question
1,Lord Eddard was the father of Arya Stark,LABEL_0,Statement


Again, the text router send the question and the statement to the expected output branches.

## Custom Use Cases for Text Classification

TransformersTextRouter is very flexible and also supports other options for classifying texts beyond distinguishing keyword queries from interrogative queries.
For example, you may be interested in detecting the sentiment of a text or classifying the topics. You can do this by loading a custom classification model from the Hugging Face Hub or by using zero-shot classification with TransformersZeroShotTextRouter.

- Traditional text classification models are trained to predict one of a few "hard-coded" classes and require a dedicated training dataset. In the Hugging Face Hub, you can find many pre-trained models, maybe even related to your domain of interest.
- Zero-shot classification is very versatile: by choosing a suitable base transformer, you can classify the text without any training dataset. You just have to provide the candidate categories.

### Custom Classification Models with TransformersTextRouter
For this use case, you can use a public model available in the Hugging Face Hub. For example, if you want to classify the sentiment of the queries, you can choose an appropriate model, such as [`cardiffnlp/twitter-roberta-base-sentiment`](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment).

In [ ]:
text_router = TransformersTextRouter(model="cardiffnlp/twitter-roberta-base-sentiment")
text_router.warm_up()

In [9]:
queries = [
    "What's the answer?",  # neutral query
    "Would you be so lovely to tell me the answer?",  # positive query
    "Can you give me the damn right answer for once??",  # negative query
]

In [10]:
sent_results = {"Query": [], "Output Branch": [], "Class": []}

for query in queries:
    result = text_router.run(text=query)
    sent_results["Query"].append(query)
    sent_results["Output Branch"].append(next(iter(result)))
    sent_results["Class"].append({"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2":"positive"}.get(next(iter(result)), "Unknown"))

pd.DataFrame.from_dict(sent_results)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,Query,Output Branch,Class
0,What's the answer?,LABEL_1,neutral
1,Would you be so lovely to tell me the answer?,LABEL_2,positive
2,Can you give me the damn right answer for once??,LABEL_0,negative


### Zero-Shot Classification with TransformersZeroShotTextRouter
TransformersZeroShotTextRouter let's you perform zero-shot classification by providing a suitable base transformer model and **defining** the classes the model should predict.

First, initialize a  TransformersZeroShotTextRouter with some custom category labels. By default, it uses the base size zero shot classification model [MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33](https://huggingface.co/MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33). You can switch to the larger model [MoritzLaurer/deberta-v3-large-zeroshot-v2.0](https://huggingface.co/MoritzLaurer/deberta-v3-large-zeroshot-v2.0) and get even better results by using `TransformersZeroShotTextRouter(model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")`.

Let's look at an example. You may be interested in whether the user query is related to music or cinema. In this case, the `labels` parameter is a list containing the candidate classes and the output branches of the component are named accordingly.

In [ ]:
from haystack.components.routers import TransformersZeroShotTextRouter

text_router = TransformersZeroShotTextRouter(labels=["music", "cinema"])
text_router.warm_up()

In [12]:
queries = [
    "In which films does John Travolta appear?",  # cinema
    "What is the Rolling Stones first album?",  # music
    "Who was Sergio Leone?",  # cinema
]

In [13]:
sent_results = {"Query": [], "Output Branch": []}

for query in queries:
    result = text_router.run(text=query)
    sent_results["Query"].append(query)
    sent_results["Output Branch"].append(next(iter(result)))

pd.DataFrame.from_dict(sent_results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Query,Output Branch
0,In which films does John Travolta appear?,cinema
1,What is the Rolling Stones first album?,music
2,Who was Sergio Leone?,cinema


Similar to the previous example, we can use zero-shot text classification to group questions into "Game of Thrones", "Star Wars" and "Lord of the Rings" related question. The number of labels is up to you!

In [14]:
from haystack.components.routers import TransformersZeroShotTextRouter

text_router = TransformersZeroShotTextRouter(labels=["Game of Thrones", "Star Wars", "Lord of the Rings"])
text_router.warm_up()

queries = [
    "Who was the father of Arya Stark",  # Game of Thrones
    "Who was the father of Luke Skywalker",  # Star Wars
    "Who was the father of Frodo Baggins",  # Lord of the Rings
]

results = {"Query": [], "Output Branch": []}

for query in queries:
    result = text_router.run(text=query)
    results["Query"].append(query)
    results["Output Branch"].append(next(iter(result)))

pd.DataFrame.from_dict(results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Query,Output Branch
0,Who was the father of Arya Stark,Game of Thrones
1,Who was the father of Luke Skywalker,Star Wars
2,Who was the father of Frodo Baggins,Lord of the Rings


And as you see, the question about "Arya Stark" is sent to branch "Game of Thrones", while the question about "Luke Skywalker" is sent to branch "Star Wars" and the question about "Frodo Baggins" is sent to "Lord of the Rings". This means you can have your pipeline treat questions about these universes differently.

Congratulations! 🎉 You’ve learned how TransformersZeroShotTextRouter and TransformersTextRouter work and how you can use these components individually. Now let's explore how to use them in a pipeline.

## Pipeline with Keyword vs. Question/Statement Query Classification

Now you will create a question-answering (QA) pipeline with keyword vs. question/statement query classification and route the questions based on the classification result.

## Fetching and Indexing Documents

You'll start creating your question answering system by downloading the data and indexing the data with its embeddings to a DocumentStore.

In this tutorial, you will take a simple approach to writing documents and their embeddings into the DocumentStore. For a full indexing pipeline with preprocessing, cleaning and splitting, check out our tutorial on [Preprocessing Different File Types](https://haystack.deepset.ai/tutorials/30_file_type_preprocessing_index_pipeline).


### Initializing the DocumentStore

Initialize a DocumentStore to index your documents. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, you'll be using the `InMemoryDocumentStore`.

In [15]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

> `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the different types of external databases that Haystack supports, see [DocumentStore Integrations](https://haystack.deepset.ai/integrations?type=Document+Store).

The DocumentStore is now ready. Now it's time to fill it with some Documents.

### Fetch the Data

You'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents. We preprocessed the data and uploaded to a Hugging Face Space: [Seven Wonders](https://huggingface.co/datasets/bilgeyucel/seven-wonders). Thus, you don't need to perform any additional cleaning or splitting.

Fetch the data and convert it into Haystack Documents:

In [ ]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

### Initalize a Document Embedder

To store your data in the DocumentStore with embeddings, initialize a [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) with the model name and call `warm_up()` to download the embedding model.

> If you'd like, you can use a different [Embedder](https://docs.haystack.deepset.ai/docs/embedders) for your documents.

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

### Write Documents to the DocumentStore

Run the `doc_embedder` with the Documents. The embedder will create embeddings for each document and save these embeddings in Document object's `embedding` field. Then, you can write the Documents to the DocumentStore with `write_documents()` method.

In [18]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

### 2) Initialize Retrievers, TextEmbedder and TransformersTextRouter

Your pipeline will be a simple Retriever pipeline, but the Retriever choice will depend on the type of query received: **keyword** queries will use a sparse BM25Retriever, while **question/statement** queries will use the more accurate but also more computationally expensive EmbeddingRetriever.

Now, initialize a Router, and Embedder, both Retrievers, and a Joiner.

In [19]:
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever, InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.joiners import DocumentJoiner

text_router = TransformersTextRouter(model="shahrukhx01/bert-mini-finetune-question-detection")
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
embedding_retriever = InMemoryEmbeddingRetriever(document_store)
bm25_retriever = InMemoryBM25Retriever(document_store)
document_joiner = DocumentJoiner()

### 3) Define the Pipeline

As promised, the question/statement branch `LABEL_0` from the text router is fed into the TextEmbedder and then the EmbeddingRetriever, while the keyword branch `LABEL_1` from the same router is fed into the BM25Retriever. Both of these retrievers are then fed into our joiner. Our pipeline can thus be thought of as having something of a diamond shape: all queries are sent into the router, which splits those queries into two different retrievers, and those retrievers feed their outputs to the same joiner.

In [20]:
from haystack import Pipeline

query_classification_pipeline = Pipeline()
query_classification_pipeline.add_component("text_router", text_router)
query_classification_pipeline.add_component("text_embedder", text_embedder)
query_classification_pipeline.add_component("embedding_retriever", embedding_retriever)
query_classification_pipeline.add_component("bm25_retriever", bm25_retriever)
query_classification_pipeline.add_component("document_joiner", document_joiner)

query_classification_pipeline.connect("text_router.LABEL_0", "text_embedder")
query_classification_pipeline.connect("text_embedder", "embedding_retriever")
query_classification_pipeline.connect("text_router.LABEL_1", "bm25_retriever")
query_classification_pipeline.connect("bm25_retriever", "document_joiner")
query_classification_pipeline.connect("embedding_retriever", "document_joiner")

🚅 Components
  - text_router: TransformersTextRouter
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
🛤️ Connections
  - text_router.LABEL_0 -> text_embedder.text (str)
  - text_router.LABEL_1 -> bm25_retriever.query (str)
  - text_embedder.embedding -> embedding_retriever.query_embedding (List[float])
  - embedding_retriever.documents -> document_joiner.documents (List[Document])
  - bm25_retriever.documents -> document_joiner.documents (List[Document])

### 4) Run the Pipeline

Below, you can see how this choice affects the branching structure: the keyword query "arya stark father" and the question query "Who is the father of Arya Stark?" generate noticeably different results, a distinction that is likely due to the use of different retrievers for keyword vs. question/statement queries.

In [21]:
# Useful for framing headers
equal_line = "=" * 30

# Run only the dense retriever on the full sentence query
res_1 = query_classification_pipeline.run({"text_router": {"text": "Who is the father of Arya Stark?"}})
print(f"\n\n{equal_line}\nQUESTION QUERY RESULTS\n{equal_line}")
print(res_1)

# Run only the sparse retriever on a keyword based query
res_2 = query_classification_pipeline.run({"text_router": {"text": "arya stark father"}})
print(f"\n\n{equal_line}\nKEYWORD QUERY RESULTS\n{equal_line}")
print(res_2)



QUESTION QUERY RESULTS
{'document_joiner': {'documents': [Document(id=4c82325818ccd91af8d68fec37108ce7a93696392f315bd0497ad3a8903d0b45, content: 'The Masonic House of the Temple of the Scottish Rite, Washington, DC, designed by John Russell Pope,...', meta: {'url': 'https://en.wikipedia.org/wiki/Mausoleum_at_Halicarnassus', '_split_id': 18}, score: 8.192663165691801, embedding: vector of size 384), Document(id=4a988f268c10bbb6af9a18063a14460b7e0126c7ed1befb2be17c9cbbc4bb064, content: 'The earliest pharaonic name of seal impressions is that of Khufu, the latest of Pepi II. Worker graf...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 4}, score: 6.652366985087608, embedding: vector of size 384), Document(id=9c1d7b92058a18bd101c037c87519e9225983c1dbb9386d51412895d5101d096, content: '[68]
The most notable account of this legend was given by Al-Masudi (896–956) in his Akbar al-zaman,...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Gi

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



KEYWORD QUERY RESULTS
{'document_joiner': {'documents': [Document(id=8d83c5906c44567371940fa0a00dfd5da94a2f0e93001c08013fa21978705df4, content: 'Conquest[edit]
In the 4th century BC, Halicarnassus was the capital of a small regional kingdom of C...', meta: {'url': 'https://en.wikipedia.org/wiki/Mausoleum_at_Halicarnassus', '_split_id': 1}, score: 0.22343484240633787), Document(id=15ba19be181b99cedeada16c282ea812d83b35f744ea5066530d40fbf784546c, content: 'Various sources describe this as a vainglorious act of arson by a man, Herostratus, who set fire to ...', meta: {'url': 'https://en.wikipedia.org/wiki/Temple_of_Artemis', '_split_id': 6}, score: 0.19531215309173927), Document(id=de0480cf304ee42ec382ba29752d16015e5b656e84a59518afe59549643e77cc, content: 'In the 7th century BC, a flood[7] destroyed the temple, depositing over half a meter of sand and flo...', meta: {'url': 'https://en.wikipedia.org/wiki/Temple_of_Artemis', '_split_id': 3}, score: 0.18115363480456034), Document(id=7737c

Above you saw a potential use for keyword vs. question/statement classification: you might choose to use a less resource-intensive retriever for keyword queries than for question/statement queries. But what about question vs. statement classification?

## Pipeline with Question vs. Statement Query Classifier

To illustrate one potential use for question vs. statement classification, you will build a pipeline that looks as follows:

1. The pipeline will start with a retriever that **every query** will go through.
2. The pipeline will end with a reader that **only question queries** will go through.

In other words, your pipeline will be a **retriever-only pipeline for statement queries**&mdash;given the statement "Arya Stark was the daughter of a Lord", all you will get back are the most relevant documents&mdash;but it will be a **retriever-reader pipeline for question queries**.

To make things more concrete, your pipeline will start with a Retriever, which is then fed into a QueryClassifier that is set to do question vs. statement classification. The QueryClassifier's first branch, which handles question queries, will then be sent to the Reader, while the second branch will not be connected to any other nodes. As a result, the last node of the pipeline depends on the type of query: **questions** go all the way through the Reader, while **statements** only go through the Retriever.

Now, define the pipeline. Keep in mind that you don't need to write Documents to the DocumentStore again as they are already indexed.

### 2) Define the Pipeline and Components

In [22]:
from haystack.components.readers import ExtractiveReader

query_classification_pipeline = Pipeline()
query_classification_pipeline.add_component("bm25_retriever_0", InMemoryBM25Retriever(document_store))
query_classification_pipeline.add_component("bm25_retriever_1", InMemoryBM25Retriever(document_store))
query_classification_pipeline.add_component("text_router", TransformersTextRouter(model="shahrukhx01/question-vs-statement-classifier"))
query_classification_pipeline.add_component("reader", ExtractiveReader())

query_classification_pipeline.connect("text_router.LABEL_0", "bm25_retriever_0")
query_classification_pipeline.connect("bm25_retriever_0", "reader")
query_classification_pipeline.connect("text_router.LABEL_1", "bm25_retriever_1")

🚅 Components
  - bm25_retriever_0: InMemoryBM25Retriever
  - bm25_retriever_1: InMemoryBM25Retriever
  - text_router: TransformersTextRouter
  - reader: ExtractiveReader
🛤️ Connections
  - bm25_retriever_0.documents -> reader.documents (List[Document])
  - text_router.LABEL_0 -> bm25_retriever_0.query (str)
  - text_router.LABEL_1 -> bm25_retriever_1.query (str)

### 2) Run the Pipeline
And here are the results of this pipeline: with a question query like "Who is the father of Arya Stark?", you obtain answers from a Reader, and with a statement query like "Arya Stark was the daughter of a Lord", you just obtain documents from a Retriever.

In [23]:
# Useful for framing headers
equal_line = "=" * 30

# Run the retriever + reader on the question query
query = "Who is the father of Arya Stark?"
res_1 = query_classification_pipeline.run({"text_router": {"text": query}, "reader": {"query": query}})
print(f"\n\n{equal_line}\nQUESTION QUERY RESULTS\n{equal_line}")
print(res_1)

# Run only the retriever on the statement query
query = "Arya Stark was the daughter of a Lord"
res_2 = query_classification_pipeline.run({"text_router": {"text": query}, "reader": {"query": query}})
print(f"\n\n{equal_line}\nKEYWORD QUERY RESULTS\n{equal_line}")
print(res_2)

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]



QUESTION QUERY RESULTS
{'bm25_retriever_1': {'documents': [Document(id=4c82325818ccd91af8d68fec37108ce7a93696392f315bd0497ad3a8903d0b45, content: 'The Masonic House of the Temple of the Scottish Rite, Washington, DC, designed by John Russell Pope,...', meta: {'url': 'https://en.wikipedia.org/wiki/Mausoleum_at_Halicarnassus', '_split_id': 18}, score: 8.192663165691801, embedding: vector of size 384), Document(id=4a988f268c10bbb6af9a18063a14460b7e0126c7ed1befb2be17c9cbbc4bb064, content: 'The earliest pharaonic name of seal impressions is that of Khufu, the latest of Pepi II. Worker graf...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 4}, score: 6.652366985087608, embedding: vector of size 384), Document(id=9c1d7b92058a18bd101c037c87519e9225983c1dbb9386d51412895d5101d096, content: '[68]
The most notable account of this legend was given by Al-Masudi (896–956) in his Akbar al-zaman,...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_G